# Install Detectron2 and its dependencies

In [ ]:
!pip install pyyaml==5.1
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html


# Imports

In [ ]:
import torch
assert torch.__version__.startswith("1.8") 
import torchvision
import cv2
from google.colab import drive

#Import some common imports
import itertools
import os
import shutil
import pandas as pd
import numpy as np
import json
import random
import copy
import matplotlib.pyplot as plt
%matplotlib inline

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.utils.visualizer import ColorMode
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import detectron2
import logging
from detectron2.utils.logger import setup_logger
setup_logger()

#imports to the validation hook
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper
import detectron2.utils.comm as comm
import time
import datetime
from detectron2.engine import launch

# Prepare the dataset

## Mount to drive

In [ ]:
drive.mount('/content/drive') #This step should be avoided if data is not placed on drive

Note that the following code has taken inspiration from different resources: The custom mapper has taken inspiration from the "VinBigData detectron2 train" kaggle. URL: https://www.kaggle.com/code/corochann/vinbigdata-detectron2-train. 
The function to convert data to coco format has taken inspiration from the "Detectron 2 compare models + augmentation" kaggle. URL:https://www.kaggle.com/code/dhiiyaur/detectron-2-compare-models-augmentation




## Make function to convert data to COCO format

In [ ]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["image_id"] = filename
        record["height"] = 480
        record["width"] = 640
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly 
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

## Make custom mapper for data augmentation

In [ ]:
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)  
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    transform_list = [
        T.RandomBrightness(0.4, 1.3),
        T.RandomCrop('relative_range', (0.6, 0.6)),
        T.RandomFlip(prob=0.4, horizontal=True, vertical=False), 
    ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict


## Make customer trainer called Trainer

In [ ]:
class Trainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper) #tells it to do augmentation


## Load data 

In [ ]:
#List the five classes 
classes = ['sink', 'door', 'bed', 'screen', 'socket']

#Give path to data - if path is not on drive change the path before running
data_path = '/content/drive/MyDrive/robotdata/Final_Dataset/'


for d in ["train", "test"]:
    DatasetCatalog.register(
        "robot_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes) #uses get_data_dicts function to make data into COCO format
    )
    MetadataCatalog.get("robot_" + d).set(thing_classes=classes)

#Collect the data into metadatacatalog 
microcontroller_metadata = MetadataCatalog.get("robot_train")

#Convert trainingset to COCO format and save as train_dicts
train_dicts = get_data_dicts(data_path+'train', classes)

#Convert testset to COCO format and save as test_dicts
test_dicts = get_data_dicts(data_path+'test', classes)

### Visualize loaded data

In [ ]:
#Visualize for checking the data is loaded properly 
for d in random.sample(train_dicts, 5):
    img = cv2.imread(d["file_name"])
    v = Visualizer(img[:, :, ::-1], metadata=microcontroller_metadata, scale=0.5)
    v = v.draw_dataset_dict(d)
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

# Final models

## R50

### Train model

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")) #Load model
cfg.DATASETS.TRAIN = ('robot_train',)
cfg.DATASETS.TEST = ()  
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml") #Load weights
cfg.SOLVER.IMS_PER_BATCH = 4 #set to four for collab to run due to limited resources
cfg.SOLVER.MAX_ITER = 3800 #iterations
cfg.SOLVER.STEPS = (500, 1000, 1500, 2000, 2500, 3000, 3500)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5 #number of classes

#Hyperparameters:
lr= 0.05
g=0.5
wd=0.00025
cfg.SOLVER.BASE_LR = lr
cfg.SOLVER.GAMMA = g
cfg.SOLVER.WEIGHT_DECAY = wg

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True) #Create directory for output
trainer= Trainer(cfg) #Use custom trainer
trainer.resume_or_load(resume=False)
trainer.train() #train model

#### Tensorboard over training

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Test final model on testset

#### Test model

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 #Set threshold
cfg.DATASETS.TEST = ("robot_test",) #give testset
predictor = DefaultPredictor(cfg) #Predict on testset 

#### Evaluation Matrices

In [ ]:

evaluator = COCOEvaluator("robot_test", output_dir="./output") #Use COCO evaluator
val_loader = build_detection_test_loader(cfg, "robot_test") #evaluate on testset
print(inference_on_dataset(predictor.model, val_loader, evaluator)) #Print evaluation

#### Visualization of predictions

In [ ]:
#Visualize all 35 images from testset
for d in random.sample(test_dicts, 35):       
    im = cv2.imread(d["file_name"])
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.5
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(14, 10))
    plt.imshow(out.get_image()[:, :, ::-1][..., ::-1])

### Save model

In [ ]:
directory=str(lr)+"_"+str(g)+"_"+str(wd)
drive='/content/drive/MyDrive'
path=os.path.join(drive,directory)
os.mkdir(path)
src="/content/output/"

for f in os.scandir(src):
  if f.is_dir():
    newpath=os.path.join(path,f.name)
    print(newpath)
    os.mkdir(newpath)
    for fi in os.scandir(f):
      shutil.copy(fi,newpath)
  elif f.is_file():
    shutil.copy(f,path)

## R101

### Train model

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")) #Load model
cfg.DATASETS.TRAIN = ('robot_train',)
cfg.DATASETS.TEST = () 
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml") #Load weights
cfg.SOLVER.IMS_PER_BATCH = 4 #Set to four so collab can run it due to limited resources
cfg.SOLVER.MAX_ITER = 3000 # iterations
cfg.SOLVER.STEPS = (1000, 2000, 3000)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5 #number of classes

#Hyperparameters:
lr= 0.07
g= 0.1
wd = 0.00025
cfg.SOLVER.BASE_LR =lr
cfg.SOLVER.GAMMA =g
cfg.SOLVER.WEIGHT_DECAY =wd

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True) #Make directory for output
trainer= Trainer(cfg)  #Use customtrainer with dataaugmentation
trainer.resume_or_load(resume=False)
trainer.train() # train model

#### Tensorboard over training

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Test final model on testset

#### Test model

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 #Set threshold
cfg.DATASETS.TEST = ("robot_test",) #give testset
predictor = DefaultPredictor(cfg) #Predict on testset 

#### Evaluation Matrices

In [ ]:

evaluator = COCOEvaluator("robot_test", output_dir="./output") #Use COCO evaluator
val_loader = build_detection_test_loader(cfg, "robot_test") #evaluate on testset
print(inference_on_dataset(predictor.model, val_loader, evaluator)) #Print evaluation

#### Visualization of predictions

In [ ]:
#Visualize all 35 images from testset
for d in random.sample(test_dicts, 35):       
    im = cv2.imread(d["file_name"])
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.5
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(14, 10))
    plt.imshow(out.get_image()[:, :, ::-1][..., ::-1])

### Save model

In [ ]:
directory=str(lr)+"_"+str(g)+"_"+str(wd)
drive='/content/drive/MyDrive'
path=os.path.join(drive,directory)
os.mkdir(path)
src="/content/output/"

for f in os.scandir(src):
  if f.is_dir():
    newpath=os.path.join(path,f.name)
    print(newpath)
    os.mkdir(newpath)
    for fi in os.scandir(f):
      shutil.copy(fi,newpath)
  elif f.is_file():
    shutil.copy(f,path)

## R50 baseline model

### Train model

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")) # Load model
cfg.DATASETS.TRAIN = ('robot_train',)
cfg.DATASETS.TEST = ('robot_valid',)  
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml") #Load weights
cfg.SOLVER.IMS_PER_BATCH = 4 #Set to four else it cannot run on collab due to resource limitations
cfg.SOLVER.MAX_ITER = 3000
cfg.SOLVER.STEPS = ()
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5 # Number of classes 

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True) #Make directory for output of model
trainer = Trainer(cfg) # Use custom trainer
trainer.resume_or_load(resume=False)
trainer.train() #Train model

#### Tensorboard over training

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Testing baseline model on testset

#### Test model

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 #Set threshold
cfg.DATASETS.TEST = ("robot_test",) #give testset
predictor = DefaultPredictor(cfg) #Predict on testset 

#### Evaluation Matrices

In [ ]:

evaluator = COCOEvaluator("robot_test", output_dir="./output") #Use COCO evaluator
val_loader = build_detection_test_loader(cfg, "robot_test") #evaluate on testset
print(inference_on_dataset(predictor.model, val_loader, evaluator)) #Print evaluation

#### Visualization of predictions

In [ ]:
#Visualize all 35 images from testset
for d in random.sample(test_dicts, 35):       
    im = cv2.imread(d["file_name"])
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.5
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(14, 10))
    plt.imshow(out.get_image()[:, :, ::-1][..., ::-1])

### Save model

In [ ]:
model = 50
iter = 3000
directory=str(model)+"_"+str(iter)
drive='/content/drive/MyDrive'
path=os.path.join(drive,directory)
os.mkdir(path)
src="/content/output/"

for f in os.scandir(src):
  if f.is_dir():
    newpath=os.path.join(path,f.name)
    print(newpath)
    os.mkdir(newpath)
    for fi in os.scandir(f):
      shutil.copy(fi,newpath)
  elif f.is_file():
    shutil.copy(f,path)

## R101 Baseline model

### Train model

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")) #Load model
cfg.DATASETS.TRAIN = ('robot_train',)
cfg.DATASETS.TEST = ()  
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml") #Load weights
cfg.SOLVER.IMS_PER_BATCH = 4 # Set to 4 to run on collab due to limited resources
cfg.SOLVER.MAX_ITER = 3000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5 #Number of classes

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True) #Make output directory 
trainer = Trainer(cfg) #Use custom trainer
trainer.resume_or_load(resume=False)
trainer.train()# train model

#### Tensorboard over training

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Testing baseline model on testset

#### Test model

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 #Set threshold
cfg.DATASETS.TEST = ("robot_test",) #give testset
predictor = DefaultPredictor(cfg) #Predict on testset 

#### Evaluation Matrices

In [ ]:

evaluator = COCOEvaluator("robot_test", output_dir="./output") #Use COCO evaluator
val_loader = build_detection_test_loader(cfg, "robot_test") #evaluate on testset
print(inference_on_dataset(predictor.model, val_loader, evaluator)) #Print evaluation

#### Visualization of predictions

In [ ]:
#Visualize all 35 images from testset
for d in random.sample(test_dicts, 35):       
    im = cv2.imread(d["file_name"])
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.5
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(14, 10))
    plt.imshow(out.get_image()[:, :, ::-1][..., ::-1])

### Save model

In [ ]:
model = 101
iter = 3000
directory=str(model)+"_"+str(iter)
drive='/content/drive/MyDrive'
path=os.path.join(drive,directory)
os.mkdir(path)
src="/content/output/"

for f in os.scandir(src):
  if f.is_dir():
    newpath=os.path.join(path,f.name)
    print(newpath)
    os.mkdir(newpath)
    for fi in os.scandir(f):
      shutil.copy(fi,newpath)
  elif f.is_file():
    shutil.copy(f,path)